This notebook combines the individual model's rating to form a unified model which performs better. The ratings from individual models are combined using a weighted linear combination to form a resultant rating. This method helps overcome the shortcomings of individual method. \\
The ratings are those generated using surprise library.

In [1]:
import pandas as pd
import numpy as np
import math

In [3]:
pred_data = pd.read_csv('../2_results/csv/test_prediction_HP.csv')
pred_data.head() 

,Unnamed: 0,uid,iid,og_rating,svd_rating,knn_rating,svdpp_rating,slopeone_rating,baseline_rating
0,0,671,51927,4.0,3.716340,4.037208,3.892101,3.926010,3.991227
1,1,671,14291,4.0,3.995477,3.960943,4.109132,4.119416,3.986223
2,2,671,10162,4.0,3.940816,4.113167,3.856270,3.868475,4.131273
3,3,671,9479,5.0,3.757214,3.273754,3.343759,3.758168,3.569950
4,4,671,8872,3.0,3.451297,3.483119,3.516192,3.635662,3.870820


In [8]:
pred_data = pred_data.rename(columns={"knn_rating": "knnwm_rating", "baseline_rating": "knn_rating"})

In [12]:
pred_data = pred_data.drop('slopeone_rating', axis = 1)

In [13]:
pred_data

,uid,iid,og_rating,svd_rating,knnwm_rating,svdpp_rating,knn_rating
0,671,51927,4.0,3.716340,4.037208,3.892101,3.991227
1,671,14291,4.0,3.995477,3.960943,4.109132,3.986223
2,671,10162,4.0,3.940816,4.113167,3.856270,4.131273
3,671,9479,5.0,3.757214,3.273754,3.343759,3.569950
4,671,8872,3.0,3.451297,3.483119,3.516192,3.870820
...,...,...,...,...,...,...,...
19990,2,197,4.0,3.594939,3.874116,3.514705,3.700537
19991,1,1598,2.0,3.083082,2.775164,2.889325,2.378814
19992,1,1051,4.0,3.002330,3.090589,2.999079,2.424992
19993,1,819,3.0,2.795546,2.471782,2.749813,2.220614


In [14]:
# num of rows:
T = pred_data.shape[0]
print(T)

19995


In [16]:
rmse = ((pred_data.og_rating - pred_data.knn_rating) ** 2).mean() ** .5
print(rmse)
mae = (((pred_data.og_rating - pred_data.knn_rating) ** 2) ** .5).mean()
print(mae)

0.9448177762755212
0.7159112913732618


In [21]:
rmse = ((pred_data.og_rating - pred_data.knnwm_rating) ** 2).mean() ** .5
print(rmse)
mae = (((pred_data.og_rating - pred_data.knnwm_rating) ** 2) ** .5).mean()
print(mae)

0.9096249508924891
0.6919726680943921


In [22]:
rmse = ((pred_data.og_rating - pred_data.svd_rating) ** 2).mean() ** .5
print(rmse)
mae = (((pred_data.og_rating - pred_data.svd_rating) ** 2) ** .5).mean()
print(mae)

0.89998713841803
0.6924169680333669


In [36]:
import math
from collections import defaultdict
import csv
from sklearn.metrics import ndcg_score
import time
from surprise.prediction_algorithms.predictions import Prediction
from surprise import accuracy

In [33]:
def get_top_n(predictions, n):
    # First map the predictions to each user.
    top_n = defaultdict(list)
    org_ratings = defaultdict(list)

    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
        org_ratings[uid].append((iid, true_r))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n, org_ratings

In [34]:
def dcg_at_k(scores):
    return scores[0] + sum(sc/math.log(ind, 2) for sc, ind in zip(scores[1:], range(2, len(scores) + 1)))

def ndcg_at_k(scores):
    idcg = dcg_at_k(sorted(scores, reverse=True))
    return (dcg_at_k(scores)/idcg) if idcg > 0.0 else 0.0

In [ ]:
def precision_recall_at_k(predictions, k=5, threshold=3.5):
    '''Return precision and recall at k metrics for each user.'''

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1

        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    precision = (sum(prec for prec in precisions.values()) / len(precisions))
    recall = (sum(rec for rec in recalls.values()) / len(recalls))

    return precision, recall

In [48]:
heso = [
    [0.05, 0.4, 0.6, 0],
    [0.4, 0.6, 0, 0],
    [0.6, 0.4, 0, 0],
    [1, 0, 0, 0],
    [0, 1, 0, 0]
]

In [49]:
for i in heso:

    svd_wt, svdpp_wt, knn_wt, knnwm_wt = i

    sqr_sum = 0
    abs_sum = 0
    
    copy = pred_data.copy()
    
    copy['new_rating'] = svd_wt*copy['svd_rating'] + knn_wt*copy['knn_rating'] + svdpp_wt*copy['svdpp_rating'] + knnwm_wt*copy['knnwm_rating']

    test_predictions = [Prediction(uid=x, iid=y, r_ui=z, est=t, details = {}) for x, y, z, t in zip(copy['uid'], copy['iid'], copy['og_rating'], copy['new_rating'])]
   
    test_rmse = accuracy.rmse(test_predictions)
    test_mae = accuracy.mae(test_predictions)
    
    top_n, org_ratings = get_top_n(test_predictions, 5)

    precision, recall = precision_recall_at_k(test_predictions)
    
    
    f_measure = (2*precision*recall)/(precision+recall)

    ndcg_scores = dict()
    for uid, user_ratings in top_n.items():
        scores = []
        for iid, est_r in user_ratings:
            iid_found = False
            org_user_ratings = org_ratings[uid]
            for i, r in org_user_ratings:
                if iid == i:
                    scores.append(r)
                    iid_found = True
                    break
            if not iid_found:
                scores.append(0)
        ndcg_scores[uid] = ndcg_at_k(scores)
    ndcg_score = sum(ndcg for ndcg in ndcg_scores.values())/len(ndcg_scores)

    print(svd_wt, svdpp_wt, knn_wt, knnwm_wt)
    print(test_rmse, test_mae, precision, recall, f_measure, ndcg_score)

RMSE: 0.9255
MAE:  0.6956
0.05 0.4 0.6 0
0.9255288045001272 0.6955589500704527 0.8131892697466483 0.43944018600324986 0.5705566675172657 0.962866469256127
RMSE: 0.8838
MAE:  0.6789
0.4 0.6 0 0
0.8838261502641134 0.6789074934949937 0.8367610531544977 0.406923284958109 0.5475626668924176 0.9643814696709648
RMSE: 0.8855
MAE:  0.6806
0.6 0.4 0 0
0.885511533349739 0.6805887020899661 0.834550422255342 0.4036504831137475 0.544122815070525 0.9632116035589675
RMSE: 0.9000
MAE:  0.6924
1 0 0 0
0.899987138418031 0.6924169680333666 0.8175111773472449 0.3969552691636696 0.5344163609962156 0.9596005540537452
RMSE: 0.8917
MAE:  0.6843
0 1 0 0
0.8916651783569112 0.6842818353796835 0.8361649279682083 0.40865732821997924 0.5490019538389129 0.965110351189141


In [39]:
pred_data

,uid,iid,og_rating,svd_rating,knnwm_rating,svdpp_rating,knn_rating
0,671,51927,4.0,3.716340,4.037208,3.892101,3.991227
1,671,14291,4.0,3.995477,3.960943,4.109132,3.986223
2,671,10162,4.0,3.940816,4.113167,3.856270,4.131273
3,671,9479,5.0,3.757214,3.273754,3.343759,3.569950
4,671,8872,3.0,3.451297,3.483119,3.516192,3.870820
...,...,...,...,...,...,...,...
19990,2,197,4.0,3.594939,3.874116,3.514705,3.700537
19991,1,1598,2.0,3.083082,2.775164,2.889325,2.378814
19992,1,1051,4.0,3.002330,3.090589,2.999079,2.424992
19993,1,819,3.0,2.795546,2.471782,2.749813,2.220614
